In [ ]:
### Ejemeplo de T-Test de una muestra
### Ejemeplo de T-Test de dos muestra
### Ejemeplo de T-Test pareado
## ANova
## Independencia de datos cualitativos

In [ ]:
## Librerias
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import math

In [ ]:
np.random.seed(1234)

In [ ]:
poblacion1=ss.poisson.rvs(loc=18,mu=35,size=450000)
poblacion2=ss.poisson.rvs(loc=18,mu=15,size=150000)

In [ ]:
Espanya=np.concatenate((poblacion1,poblacion2))

In [ ]:
granada1=ss.poisson.rvs(loc=18,mu=34,size=4500)
granada2=ss.poisson.rvs(loc=18,mu=14,size=1500)
Granada=np.concatenate((granada1,granada2))

In [ ]:
print(Espanya.mean())
print(Granada.mean())

$$H_0 :   \mu_{G}=\mu_{ES}$$
$$H_1 : \mu_{G} \neq \mu_{ES}$$

In [ ]:
estadistico,pvalor=ss.ttest_1samp(Granada,Espanya.mean())

In [ ]:
if pvalor<=0.05:
    print("Rechazamos la ****")
else:
    print("Aceptamos la ***")

In [ ]:
Madrid1=ss.poisson.rvs(loc=18,mu=37,size=120)
Madrid2=ss.poisson.rvs(loc=18,mu=13,size=80)
Madrid=np.concatenate((Madrid1,Madrid2))

In [ ]:
Madrid.mean()

In [ ]:
ss.ttest_ind(Granada,Madrid,equal_var=False)

$$ H_0 : \mu_{Gr} \leq \mu_{Ma}$$
$$ H_1 : \mu_{Gr} > \mu_{Ma}$$

In [ ]:
ss.ttest_ind(Granada,Madrid,equal_var=False,alternative="greater")

In [ ]:
## T-test Pareado
peso_precovid=ss.norm.rvs(scale=30,loc=100,size=1000)
peso_postcovid=peso_precovid+ss.norm.rvs(scale=10,loc=1.25,size=1000)

In [ ]:
Pesos=pd.DataFrame({"peso_precovid":peso_precovid,
              "peso_postcovid":peso_postcovid,
              "Diferencia":peso_postcovid-peso_precovid})

In [ ]:
Pesos.info()

In [ ]:
Pesos.describe().T

$$ H_0: \mu_{AC} = \mu_{DC} $$
$$ H_1: \mu_{AC} \neq \mu_{DC} $$

In [ ]:
ss.ttest_rel(peso_precovid,peso_postcovid,alternative="two-sided")

$$ H_0: \mu_{AC} \leq \mu_{DC} $$
$$ H_1: \mu_{AC} > \mu_{DC} $$

In [ ]:
ss.ttest_rel(peso_precovid,peso_postcovid,alternative="greater")

$$ H_0: \mu_{AC} \geq \mu_{DC} $$
$$ H_1: \mu_{AC} < \mu_{DC} $$

In [ ]:
ss.ttest_rel(peso_precovid,peso_postcovid,alternative="less")

In [ ]:
ss.ttest_rel?

### Anova

In [ ]:
Peliculas=["IJ","Bat","Marvel","Senor_Anillos","Tenet"]
pelicula_vista=np.random.choice(a=Peliculas,
                                p=[0.1,0.2,0.2,0.15,0.35],
                                size=1000)
voto_Edad=ss.poisson.rvs(loc=9,mu=25,size=1000)

In [ ]:
df=pd.DataFrame({"Pelicula":pelicula_vista,
              "Edad":voto_Edad})

In [ ]:
df

In [ ]:
grupos=df.groupby('Pelicula').groups

In [ ]:
Bat=voto_Edad[grupos["Bat"]]
IJ=voto_Edad[grupos["IJ"]]
Marvel=voto_Edad[grupos["Marvel"]]
Tenet=voto_Edad[grupos["Tenet"]]
Senor_Anillos=voto_Edad[grupos["Senor_Anillos"]]

In [ ]:
ss.f_oneway(Senor_Anillos,Bat,Tenet,IJ,Marvel)

$$ H_0: \mu_{0}=\mu_{1}=\ldots=\mu_{n}$$
$$ H_1: \mu_{i}\neq\mu_{j} \text{existe un i que la media es diferente para un j}$$

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
modelo=ols('Edad~Pelicula',data=df).fit()

In [ ]:
modelo

In [ ]:
sm.stats.anova_lm(modelo,typ=2)

In [ ]:
IJ_Edad=ss.poisson.rvs(loc=18,mu=35,size=1000)
voto_Edad=np.where(pelicula_vista=="IJ",IJ_Edad,voto_Edad)

In [ ]:
df_1=pd.DataFrame({"Pelicula":pelicula_vista,
              "Edad":voto_Edad})

In [ ]:
modelo=ols('Edad~Pelicula',data=df_1).fit()

In [ ]:
sm.stats.anova_lm(modelo,typ=2)

### Ejercicio
De manera automática encontrar el grupo diferente

### Solución
* Hacer las parejas
    *  Crearé una lista vacía para guardar las parejas
    * Lo haré mediante un for y tengo que evitar que las parejas se repitan o sean de la misma pelicula
* Cada pareja pasará por un ttest_ind
* Supondré que la varianza es desigual
* Si el p>0.05 pasaré
* Si es P<0.05 me quedaré con esa pareja



In [ ]:
Peliculas=["IJ","Bat","Marvel","Senor_Anillos","Tenet"]
## Hacemos parejas
Parejas=[]
for x in range(4):
    for y in range(x+1,5):
        Parejas.append((Peliculas[x],Peliculas[y]))

In [ ]:
Parejas

In [ ]:
Parejas[0][0]

In [ ]:
ss.ttest_ind(voto_Edad[grupos[Parejas[0][0]]],
             voto_Edad[grupos[Parejas[0][1]]],
             equal_var=False)

In [ ]:
for Peli1,Peli2 in Parejas:
    estadistico,pvalor=ss.ttest_ind(voto_Edad[grupos[Peli1]],
             voto_Edad[grupos[Peli2]],
             equal_var=False)
    if pvalor<0.05:
        print("\n",Peli1,"-vs-",Peli2)

In [ ]:
from  statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
tukey=pairwise_tukeyhsd(voto_Edad,pelicula_vista,alpha=0.05)

In [ ]:
tukey.summary()

In [ ]:
tukey.plot_simultaneous()